In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the dataset, skipping the problematic header rows
try:
    df = pd.read_csv('nifty50_index_data_2010_2023.csv', header=2)
    # The date is in the first column, let's rename it and set it as the index
    df.rename(columns={df.columns[0]: 'Date'}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # The column names might have been read from the wrong row, let's see the columns
    # The actual data seems to have columns Open, High, Low, Close, Adj Close, Volume
    # The file has Price,Close,High,Low,Open,Volume
    # Let's check the columns and rename if necessary
    # From the file preview, the columns are messed up. Let's fix them.
    # The yfinance download gives 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'
    # The saved file has a weird structure.
    # Let's try to load it in a more robust way.
    
    # Reloading with better parameters
    df = pd.read_csv('nifty50_index_data_2010_2023.csv', index_col=0, parse_dates=True, header=[0,1])
    # This creates a multi-index header, let's flatten it
    df.columns = df.columns.get_level_values(0)
    # Now the columns should be 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'
    # Let's drop Adj Close as it's not used in the plots and can be confusing.
    if 'Adj Close' in df.columns:
        df = df.drop(columns=['Adj Close'])

    print("Data loaded successfully.")
    display(df.head())
except FileNotFoundError:
    print("Error: 'nifty50_index_data_2010_2023.csv' not found. Make sure the file is in the same directory as the notebook.")
    df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred: {e}")
    df = pd.DataFrame()

Data loaded successfully.


Price,Close,High,Low,Open,Volume
Date,,,,,
2010-01-04,5232.200195,5238.450195,5167.100098,5200.899902,0
2010-01-05,5277.899902,5288.350098,5242.399902,5277.149902,0
2010-01-06,5281.799805,5310.850098,5260.049805,5278.149902,0
2010-01-07,5263.100098,5302.549805,5244.750000,5281.799805,0
2010-01-08,5244.750000,5276.750000,5234.700195,5264.250000,0


### 1. Price Trend Over Time
This chart shows the closing price of the Nifty 50 index over the selected date range. It helps visualize the overall trend (upward, downward, or sideways) of the market.

In [2]:
if not df.empty:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'))
    fig.update_layout(
        title='Nifty 50 Closing Price Trend (2010-2023)',
        xaxis_title='Date',
        yaxis_title='Price (INR)',
        template='plotly_dark'
    )
    fig.show()

### 2. OHLC Chart
An OHLC (Open, High, Low, Close) chart is a type of financial chart that shows the open, high, low, and close prices for a given period. It's useful for understanding the price volatility and momentum in a single view.

In [3]:
if not df.empty:
    # Resample to monthly data for a cleaner OHLC plot
    df_monthly = df.resample('M').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})
    
    fig = go.Figure(data=[go.Ohlc(
        x=df_monthly.index,
        open=df_monthly['Open'],
        high=df_monthly['High'],
        low=df_monthly['Low'],
        close=df_monthly['Close']
    )])
    
    fig.update_layout(
        title='Nifty 50 OHLC Chart (Monthly)',
        xaxis_title='Date',
        yaxis_title='Price (INR)',
        xaxis_rangeslider_visible=False,
        template='plotly_dark'
    )
    fig.show()

C:\Users\amart\AppData\Local\Temp\ipykernel_13096\3814244380.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



### 3. Moving Averages (MA)
Moving averages smooth out price data to create a single flowing line, making it easier to identify the direction of the trend. We'll plot the 50-day and 200-day moving averages, which are common indicators for short-term and long-term trends, respectively.

In [4]:
if not df.empty:
    # Calculate moving averages
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price', line=dict(color='lightblue', width=1)))
    fig.add_trace(go.Scatter(x=df.index, y=df['MA50'], mode='lines', name='50-Day MA', line=dict(color='orange', width=2)))
    fig.add_trace(go.Scatter(x=df.index, y=df['MA200'], mode='lines', name='200-Day MA', line=dict(color='red', width=2)))
    
    fig.update_layout(
        title='Nifty 50 with 50-Day and 200-Day Moving Averages',
        xaxis_title='Date',
        yaxis_title='Price (INR)',
        template='plotly_dark'
    )
    fig.show()

### 4. Relative Strength Index (RSI)
The RSI is a momentum oscillator that measures the speed and change of price movements. It oscillates between 0 and 100. Traditionally, an RSI above 70 is considered overbought, and an RSI below 30 is considered oversold.

In [5]:
if not df.empty:
    # Manual RSI Calculation
    def calculate_rsi(data, window=14):
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    df['RSI'] = calculate_rsi(df)

    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=('Nifty 50 Close Price', 'Relative Strength Index (RSI)'))

    # Plot Close Price
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'), row=1, col=1)

    # Plot RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], mode='lines', name='RSI'), row=2, col=1)

    # Add overbought and oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, name="Overbought")
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, name="Oversold")

    fig.update_layout(
        title_text='Nifty 50 Price and RSI',
        template='plotly_dark',
        height=700
    )
    fig.update_yaxes(title_text="Price (INR)", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1)
    
    fig.show()